## PARCEIROS 360 - COMISSAO

In [8]:
!python --version
!numpy --version

Python 3.8.16


'numpy' n�o � reconhecido como um comando interno
ou externo, um programa oper�vel ou um arquivo em lotes.


### BIBLIOTECAS:

In [9]:
## data
import pandas as pd
import numpy as np
import datetime as dt
from dateutil.relativedelta import relativedelta
from unicodedata import normalize

## vis
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

## modelo
from sklearn import preprocessing
import shap

## alertas
import warnings
warnings.filterwarnings("ignore")

from sklearn.cluster import KMeans
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler

from treeinterpreter import treeinterpreter as ti

### LAYOUT:

In [10]:
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (12, 6)

pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 1000)

rc={'font.size': 16, 'axes.labelsize': 16, 'legend.fontsize': 16,
    'axes.titlesize': 18, 'xtick.labelsize': 16, 'ytick.labelsize': 16}

sns.set(rc=rc)
sns.set_style("darkgrid")
sns.set_palette("pastel")

### CONFIGS:

In [11]:
#pd.set_option('max_columns', 200)
pd.options.display.max_columns = 200 

seed = 42
np.random.seed(seed)

### DADOS:

In [12]:
PATH = "datasets/"

In [13]:
FILE = "ALIADOS_KPI.csv"
original = pd.read_csv(PATH + FILE, sep = ';', encoding = 'ISO-8859-1', engine ='python')
dados = original.copy()
dados.set_index(['RAIZ_CNPJ', 'COMPETENCIA', 'NOME_FORNECEDOR'], inplace=True)
dados

,,,HCS_SAP_GT,HCS,PERC_SERVICO,QDE_PROC_TRAB,ATIVA_FGTS,PARCELADA_FGTS,FGTS_TOTAL,ATIVA_INSS,PARCELADA_INSS,INSS_TOTAL,TOTAL_GERAL,PENDENCIA_NAO_CONFORME,MONITORADO_SIM,MONITORADO_NAO,MM6,KPI_A1_variacao,KPI_A2_trabalhista,KPI_A3_endividamento
RAIZ_CNPJ,COMPETENCIA,NOME_FORNECEDOR,,,,,,,,,,,,,,,,,,
00000000,2021-01-01,HODNET S/A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0.000000,BOM,BOM,BOM
00000B60,2021-01-01,ITP TECNOLOGICA SL,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0.000000,BOM,BOM,BOM
00000B86,2021-01-01,"DELOITTE ADVISORY, S.L.",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0.000000,BOM,BOM,BOM
00028986,2021-01-01,ELEVADORES ATLAS SCHINDLER SA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0.000000,BOM,BOM,BOM
00059799,2021-01-01,HDS SISTEMAS DE ENERGIA LTDA,22.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0.000000,BOM,BOM,BOM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96662168,2022-07-01,DREBES E CIA LTDA,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,1.000000,RUIM,BOM,BOM
98102924,2022-07-01,DELTASUL UTILIDADES LTDA,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0.000000,BOM,BOM,BOM
B8677373,2022-07-01,"NETMETRIX SOLUTIONS, S.L.",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0.833333,BOM,BOM,BOM


In [17]:
base_treino_KPI_A3_endividamento = dados[['ATIVA_FGTS', 'PARCELADA_FGTS', 'FGTS_TOTAL', 'ATIVA_INSS', 'PARCELADA_INSS', 'INSS_TOTAL', 'TOTAL_GERAL']]
base_treino_KPI_A2_trabalhista = dados[['QDE_PROC_TRAB', 'HCS_SAP_GT', 'HCS', 'MONITORADO_SIM']]
base_treino_KPI_A1_variacao = dados[['MM6', 'HCS_SAP_GT', 'HCS', 'MONITORADO_SIM']]

In [18]:
base_treino_KPI_A1_variacao

,,,MM6,HCS_SAP_GT,HCS,MONITORADO_SIM
RAIZ_CNPJ,COMPETENCIA,NOME_FORNECEDOR,,,,
00000000,2021-01-01,HODNET S/A,0.000000,0.0,0.0,0
00000B60,2021-01-01,ITP TECNOLOGICA SL,0.000000,3.0,0.0,0
00000B86,2021-01-01,"DELOITTE ADVISORY, S.L.",0.000000,1.0,0.0,0
00028986,2021-01-01,ELEVADORES ATLAS SCHINDLER SA,0.000000,0.0,0.0,0
00059799,2021-01-01,HDS SISTEMAS DE ENERGIA LTDA,0.000000,22.0,0.0,0
...,...,...,...,...,...,...
96662168,2022-07-01,DREBES E CIA LTDA,1.000000,1.0,0.0,0
98102924,2022-07-01,DELTASUL UTILIDADES LTDA,0.000000,1.0,0.0,0
B8677373,2022-07-01,"NETMETRIX SOLUTIONS, S.L.",0.833333,0.0,0.0,0


In [19]:
base_treino_KPI_A2_trabalhista

,,,QDE_PROC_TRAB,HCS_SAP_GT,HCS,MONITORADO_SIM
RAIZ_CNPJ,COMPETENCIA,NOME_FORNECEDOR,,,,
00000000,2021-01-01,HODNET S/A,0.0,0.0,0.0,0
00000B60,2021-01-01,ITP TECNOLOGICA SL,0.0,3.0,0.0,0
00000B86,2021-01-01,"DELOITTE ADVISORY, S.L.",0.0,1.0,0.0,0
00028986,2021-01-01,ELEVADORES ATLAS SCHINDLER SA,0.0,0.0,0.0,0
00059799,2021-01-01,HDS SISTEMAS DE ENERGIA LTDA,0.0,22.0,0.0,0
...,...,...,...,...,...,...
96662168,2022-07-01,DREBES E CIA LTDA,0.0,1.0,0.0,0
98102924,2022-07-01,DELTASUL UTILIDADES LTDA,0.0,1.0,0.0,0
B8677373,2022-07-01,"NETMETRIX SOLUTIONS, S.L.",0.0,0.0,0.0,0


In [20]:
base_treino_KPI_A3_endividamento

,,,ATIVA_FGTS,PARCELADA_FGTS,FGTS_TOTAL,ATIVA_INSS,PARCELADA_INSS,INSS_TOTAL,TOTAL_GERAL
RAIZ_CNPJ,COMPETENCIA,NOME_FORNECEDOR,,,,,,,
00000000,2021-01-01,HODNET S/A,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00000B60,2021-01-01,ITP TECNOLOGICA SL,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00000B86,2021-01-01,"DELOITTE ADVISORY, S.L.",0.0,0.0,0.0,0.0,0.0,0.0,0.0
00028986,2021-01-01,ELEVADORES ATLAS SCHINDLER SA,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00059799,2021-01-01,HDS SISTEMAS DE ENERGIA LTDA,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
96662168,2022-07-01,DREBES E CIA LTDA,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98102924,2022-07-01,DELTASUL UTILIDADES LTDA,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B8677373,2022-07-01,"NETMETRIX SOLUTIONS, S.L.",0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
#SE PRECISAR DO ONE HOT USAR ANTES DO SCALERS PRA NAO DAR ERRO

def scalers(base):
    nova_base = base.copy()
    scaler = MinMaxScaler(feature_range=(0,1))
    for column in base.columns:
        aux = scaler.fit_transform(base[[column]])
        nova_base[column] = aux
    return nova_base

In [22]:
base_treino_KPI_A1_variacao_scaled = scalers(base_treino_KPI_A1_variacao)
base_treino_KPI_A2_trabalhista_scaled = scalers(base_treino_KPI_A2_trabalhista)
base_treino_KPI_A3_endividamento_scaled = scalers(base_treino_KPI_A3_endividamento)

In [23]:
base_treino_KPI_A3_endividamento.head(3)

,,,ATIVA_FGTS,PARCELADA_FGTS,FGTS_TOTAL,ATIVA_INSS,PARCELADA_INSS,INSS_TOTAL,TOTAL_GERAL
RAIZ_CNPJ,COMPETENCIA,NOME_FORNECEDOR,,,,,,,
00000000,2021-01-01,HODNET S/A,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00000B60,2021-01-01,ITP TECNOLOGICA SL,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00000B86,2021-01-01,"DELOITTE ADVISORY, S.L.",0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
base_treino_KPI_A2_trabalhista.head(3)

,,,QDE_PROC_TRAB,HCS_SAP_GT,HCS,MONITORADO_SIM
RAIZ_CNPJ,COMPETENCIA,NOME_FORNECEDOR,,,,
00000000,2021-01-01,HODNET S/A,0.0,0.0,0.0,0
00000B60,2021-01-01,ITP TECNOLOGICA SL,0.0,3.0,0.0,0
00000B86,2021-01-01,"DELOITTE ADVISORY, S.L.",0.0,1.0,0.0,0


In [25]:
base_treino_KPI_A1_variacao.head(3)

,,,MM6,HCS_SAP_GT,HCS,MONITORADO_SIM
RAIZ_CNPJ,COMPETENCIA,NOME_FORNECEDOR,,,,
00000000,2021-01-01,HODNET S/A,0.0,0.0,0.0,0
00000B60,2021-01-01,ITP TECNOLOGICA SL,0.0,3.0,0.0,0
00000B86,2021-01-01,"DELOITTE ADVISORY, S.L.",0.0,1.0,0.0,0


### CLUSTERING COM K-Means:

In [26]:
#base_treino_scaled = BASE DE TREINO, SEM A COLUNA DO KPI, scaled
#DADOS = BASE COMPLETA, COM O TARGET
#KPI NOME = O NOME DA COLUNA ONDE ESTÁ O TARGET
#base_treino = BASE DE TREINO, SEM A COLUNA DO KPI

def makedf(base_treino_scaled, dados, kpi_nome, base_treino):
    
    kmeans = KMeans(n_clusters = 3, max_iter = 300, init = 'k-means++', n_init = 10, random_state = 0)
    clusters = kmeans.fit_predict(base_treino_scaled)

    model = RandomForestClassifier(random_state=0)
    model.fit(base_treino_scaled,clusters)
    prediction, bias, contributions = ti.predict(model, base_treino_scaled)

    contributions = np.round(contributions,5)*1000

    contributions_c1 = []
    contributions_c2 = []
    contributions_c3 = []
    contributions_filtered = []

    if kpi_nome == 'KPI_A3_endividamento':
        for i in range(len(contributions)):
            contributions_c1.append([contributions[i][0][0], contributions[i][1][0], contributions[i][2][0], contributions[i][3][0], contributions[i][4][0], contributions[i][5][0], contributions[i][6][0]])
            contributions_c2.append([contributions[i][0][1], contributions[i][1][1], contributions[i][2][1], contributions[i][3][1], contributions[i][4][1], contributions[i][5][1], contributions[i][6][1]])
            contributions_c3.append([contributions[i][0][2], contributions[i][1][2], contributions[i][2][2], contributions[i][3][2], contributions[i][4][2], contributions[i][5][2], contributions[i][6][2]])
    else:
        for i in range(len(contributions)):
            contributions_c1.append([contributions[i][0][0], contributions[i][1][0], contributions[i][2][0], contributions[i][3][0]])
            contributions_c2.append([contributions[i][0][1], contributions[i][1][1], contributions[i][2][1], contributions[i][3][1]])
            contributions_c3.append([contributions[i][0][2], contributions[i][1][2], contributions[i][2][2], contributions[i][3][2]])

    for i in range(len(clusters)):
        if clusters[i] == 0:
            contributions_filtered.append(contributions_c1[i])
        if clusters[i] == 1:
            contributions_filtered.append(contributions_c2[i])
        if clusters[i] == 2:
            contributions_filtered.append(contributions_c3[i])

    scaler = MinMaxScaler(feature_range=(0,100))
    contributions_filtered_scaled = scaler.fit_transform(contributions_filtered)

    contributions_filtered = pd.DataFrame(contributions_filtered_scaled, columns=base_treino_scaled.columns + '_CONTRIBUTION')

    base_treino['KPI_original'] = dados[kpi_nome].values
    base_treino['resultado_modelo'] = clusters
    base_treino['resultado_modelo'] = base_treino['resultado_modelo'].map({0:'BOM', 1:'MEDIO', 2:'RUIM'})

    base_treino.reset_index(inplace=True, drop=False)

    results = pd.concat([base_treino, contributions_filtered], axis=1)
    return results

In [27]:
results_kpi1 = makedf(base_treino_KPI_A1_variacao_scaled, dados, 'KPI_A1_variacao', base_treino_KPI_A1_variacao)
results_kpi1

,RAIZ_CNPJ,COMPETENCIA,NOME_FORNECEDOR,MM6,HCS_SAP_GT,HCS,MONITORADO_SIM,KPI_original,resultado_modelo,MM6_CONTRIBUTION,HCS_SAP_GT_CONTRIBUTION,HCS_CONTRIBUTION,MONITORADO_SIM_CONTRIBUTION
0,00000000,2021-01-01,HODNET S/A,0.000000,0.0,0.0,0,BOM,BOM,29.274827,17.039350,16.05737,8.530902
1,00000B60,2021-01-01,ITP TECNOLOGICA SL,0.000000,3.0,0.0,0,BOM,BOM,29.275751,17.046489,16.05737,8.525273
2,00000B86,2021-01-01,"DELOITTE ADVISORY, S.L.",0.000000,1.0,0.0,0,BOM,BOM,29.274827,17.028643,16.05737,8.536530
3,00028986,2021-01-01,ELEVADORES ATLAS SCHINDLER SA,0.000000,0.0,0.0,0,BOM,BOM,29.274827,17.039350,16.05737,8.530902
4,00059799,2021-01-01,HDS SISTEMAS DE ENERGIA LTDA,0.000000,22.0,0.0,0,BOM,BOM,29.272979,16.859106,16.05737,8.647979
...,...,...,...,...,...,...,...,...,...,...,...,...,...
25786,96662168,2022-07-01,DREBES E CIA LTDA,1.000000,1.0,0.0,0,RUIM,BOM,29.278522,17.039350,16.05737,8.525273
25787,98102924,2022-07-01,DELTASUL UTILIDADES LTDA,0.000000,1.0,0.0,0,BOM,BOM,29.274827,17.028643,16.05737,8.536530
25788,B8677373,2022-07-01,"NETMETRIX SOLUTIONS, S.L.",0.833333,0.0,0.0,0,BOM,BOM,29.279446,17.037566,16.05737,8.526399
25789,B9898262,2022-07-01,"KENMEI TECHNOLOGIES, S.L.",0.000000,0.0,0.0,0,BOM,BOM,29.274827,17.039350,16.05737,8.530902


In [28]:
results_kpi2 = makedf(base_treino_KPI_A2_trabalhista_scaled, dados, 'KPI_A2_trabalhista', base_treino_KPI_A2_trabalhista)
results_kpi2

,RAIZ_CNPJ,COMPETENCIA,NOME_FORNECEDOR,QDE_PROC_TRAB,HCS_SAP_GT,HCS,MONITORADO_SIM,KPI_original,resultado_modelo,QDE_PROC_TRAB_CONTRIBUTION,HCS_SAP_GT_CONTRIBUTION,HCS_CONTRIBUTION,MONITORADO_SIM_CONTRIBUTION
0,00000000,2021-01-01,HODNET S/A,0.0,0.0,0.0,0,BOM,BOM,36.808382,30.228603,20.940570,9.166327
1,00000B60,2021-01-01,ITP TECNOLOGICA SL,0.0,3.0,0.0,0,BOM,BOM,36.808382,30.228603,20.952018,9.158405
2,00000B86,2021-01-01,"DELOITTE ADVISORY, S.L.",0.0,1.0,0.0,0,BOM,BOM,36.808382,30.228603,20.940570,9.165195
3,00028986,2021-01-01,ELEVADORES ATLAS SCHINDLER SA,0.0,0.0,0.0,0,BOM,BOM,36.808382,30.228603,20.940570,9.166327
4,00059799,2021-01-01,HDS SISTEMAS DE ENERGIA LTDA,0.0,22.0,0.0,0,BOM,BOM,36.808382,29.849233,20.940570,9.297614
...,...,...,...,...,...,...,...,...,...,...,...,...,...
25786,96662168,2022-07-01,DREBES E CIA LTDA,0.0,1.0,0.0,0,BOM,BOM,36.808382,30.228603,20.940570,9.165195
25787,98102924,2022-07-01,DELTASUL UTILIDADES LTDA,0.0,1.0,0.0,0,BOM,BOM,36.808382,30.228603,20.940570,9.165195
25788,B8677373,2022-07-01,"NETMETRIX SOLUTIONS, S.L.",0.0,0.0,0.0,0,BOM,BOM,36.808382,30.228603,20.940570,9.166327
25789,B9898262,2022-07-01,"KENMEI TECHNOLOGIES, S.L.",0.0,0.0,0.0,0,BOM,BOM,36.808382,30.228603,20.940570,9.166327


In [29]:
results_kpi3 = makedf(base_treino_KPI_A3_endividamento_scaled, dados, 'KPI_A3_endividamento', base_treino_KPI_A3_endividamento)
results_kpi3

,RAIZ_CNPJ,COMPETENCIA,NOME_FORNECEDOR,ATIVA_FGTS,PARCELADA_FGTS,FGTS_TOTAL,ATIVA_INSS,PARCELADA_INSS,INSS_TOTAL,TOTAL_GERAL,KPI_original,resultado_modelo,ATIVA_FGTS_CONTRIBUTION,PARCELADA_FGTS_CONTRIBUTION,FGTS_TOTAL_CONTRIBUTION,ATIVA_INSS_CONTRIBUTION,PARCELADA_INSS_CONTRIBUTION,INSS_TOTAL_CONTRIBUTION,TOTAL_GERAL_CONTRIBUTION
0,00000000,2021-01-01,HODNET S/A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BOM,BOM,35.156154,25.763728,25.861899,44.315216,38.315718,40.000581,51.11947
1,00000B60,2021-01-01,ITP TECNOLOGICA SL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BOM,BOM,35.156154,25.763728,25.861899,44.315216,38.315718,40.000581,51.11947
2,00000B86,2021-01-01,"DELOITTE ADVISORY, S.L.",0.0,0.0,0.0,0.0,0.0,0.0,0.0,BOM,BOM,35.156154,25.763728,25.861899,44.315216,38.315718,40.000581,51.11947
3,00028986,2021-01-01,ELEVADORES ATLAS SCHINDLER SA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BOM,BOM,35.156154,25.763728,25.861899,44.315216,38.315718,40.000581,51.11947
4,00059799,2021-01-01,HDS SISTEMAS DE ENERGIA LTDA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BOM,BOM,35.156154,25.763728,25.861899,44.315216,38.315718,40.000581,51.11947
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25786,96662168,2022-07-01,DREBES E CIA LTDA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BOM,BOM,35.156154,25.763728,25.861899,44.315216,38.315718,40.000581,51.11947
25787,98102924,2022-07-01,DELTASUL UTILIDADES LTDA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BOM,BOM,35.156154,25.763728,25.861899,44.315216,38.315718,40.000581,51.11947
25788,B8677373,2022-07-01,"NETMETRIX SOLUTIONS, S.L.",0.0,0.0,0.0,0.0,0.0,0.0,0.0,BOM,BOM,35.156154,25.763728,25.861899,44.315216,38.315718,40.000581,51.11947
25789,B9898262,2022-07-01,"KENMEI TECHNOLOGIES, S.L.",0.0,0.0,0.0,0.0,0.0,0.0,0.0,BOM,BOM,35.156154,25.763728,25.861899,44.315216,38.315718,40.000581,51.11947


In [31]:
results_kpi1.to_excel(r'outputs/previsao_aliados_variacao_kpi1.xlsx')
results_kpi2.to_excel(r'outputs/previsao_aliados_trabalhista_kpi2.xlsx')
results_kpi3.to_excel(r'outputs/previsao_aliados_endividamento_kpi3.xlsx')